# Loading python libraries

In [ ]:
import pandas as pd
import plotly.express as px

# Loading Data

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv")

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

train.info()

In [ ]:
# train.dropna(subset=['Age', 'Fare', 'Sex', 'Embarked'], inplace=True)

In [ ]:
train.corr()

# Data Exploration

In [ ]:
fig = px.density_heatmap(train, x="Age", y="Embarked", z='Survived',nbinsx=50, facet_row="Sex", marginal_y='histogram')
fig.show()

In [ ]:
fig = px.histogram(train, x="Age", color="Survived", marginal="rug", facet_row='Sex', nbins=50)
fig.show()

In [ ]:
fig = px.density_heatmap(train, x="Age", y="Pclass", z='Survived',nbinsx=50, facet_row="Sex", marginal_y='histogram')
fig.show()

# Building ml model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:
model = RandomForestClassifier()

In [ ]:
def clean_values_of_Sex(x):
    if x == 'male':
        return 0
    elif x == 'female':
        return 1
    else:
        return 2

def clean_vales_of_Embarked(x):
    if x == 'S':
        return 0
    elif x == 'C':
        return 1
    elif x == 'Q':
        return 2
    else:
        return 3

def clean_values_of_Fare(x):
    if np.isnan(x):
        return -1
    else:
        return x

def clean_values_of_Age(x):
    if np.isnan(x):
        return -1
    else:
        return x

train['Embarked'] = train['Embarked'].apply(clean_vales_of_Embarked)
train['Sex'] = train['Sex'].apply(clean_values_of_Sex)
train['Fare'] = train['Fare'].apply(clean_values_of_Fare)
train['Age'] = train['Age'].apply(clean_values_of_Age)

test['Embarked'] = test['Embarked'].apply(clean_vales_of_Embarked)
test['Sex'] = test['Sex'].apply(clean_values_of_Sex)
test['Fare'] = test['Fare'].apply(clean_values_of_Fare)
test['Age'] = test['Age'].apply(clean_values_of_Age)

In [ ]:
train = train.iloc[:-10000]
val = train.iloc[-10000:]

In [ ]:
X = []
for (a, b, c, d, e) in zip(train['Pclass'], train['Embarked'], train['Age'], train['Fare'], train['Sex']):
    X.append([a, b, c, d, e])
    
y = np.array(train['Survived'])
y.reshape(1,-1)

model.fit(X=X,y=y)

### Scoring with created validation dataset

In [ ]:
X2 = []
for (a, b, c, d, e) in zip(val['Pclass'], val['Embarked'], val['Age'], val['Fare'], val['Sex']):
    X2.append([a, b, c, d, e])

y2 = np.array(val['Survived'])
y2.reshape(1,-1)
y2.shape

In [ ]:
model.score(X2,y2)

In [ ]:
import time

In [ ]:
y = []
r = 0
since = time.time()
for idd in df['PassengerId']:
    
    row = test.loc[test['PassengerId'] == idd]
    pclass = int(row['Pclass'])
    embarked = int(row['Embarked'])
    age = float(row['Age'])
    fare = float(row['Fare'])
    sex = int(row['Sex'])
    x = [[pclass, embarked, age, fare, sex]]
    df['Survived'][r] = model.predict(X=x)
    if (r % 10000) == 0:
        
        now = time.time()
        print("Finished up to row: {}".format(r))
        print("Time Elapsed Since last batch: {}".format(now - since))
        since = time.time()
    r += 1

In [ ]:
df

In [ ]:
df.to_csv("RandomForestClassifier.csv", index=False)

# I achieved accuracy of 71.906%